In [ ]:
import seaborn as sns

import json
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib
import cv2
import os



In [ ]:

path = "../input/iwildcam2021-fgvc8/"
with open(path+'metadata/'+'iwildcam2021_train_annotations.json') as f:
    train_data = json.load(f)
with open(path+'metadata/'+'iwildcam2021_test_information.json') as f:
    test_data = json.load(f)
with open(path+'metadata/'+'iwildcam2021_megadetector_results.json') as f:
    results_data = json.load(f)



In [ ]:
test_data.keys()

In [ ]:
df_test_image = pd.json_normalize(test_data['images'])
df_test_image.set_index('id',inplace=True)

In [ ]:
train_data.keys()

In [ ]:
df_categories = pd.json_normalize(train_data['categories'])
df_categories.set_index('id',inplace=True)
df_categories.head()

In [ ]:
df_train_annotation = pd.json_normalize(train_data['annotations'])
df_train_annotation.set_index('id',inplace=True)
df_train_annotation['category_name'] = df_train_annotation['category_id'].apply(lambda x: df_categories['name'][x] if x in df_categories.index else 'Unkown' )
df_train_annotation.head()

In [ ]:
df_train_image_data = pd.json_normalize(train_data['images'])
df_train_image_data.set_index('id',inplace=True)
df_train_image_data.head()

In [ ]:
df_train_merge = pd.merge(df_train_image_data, df_train_annotation, left_on='id', right_on='image_id')
df_train_merge.set_index('image_id',inplace=True)
df_train_merge.head()

In [ ]:
result_image_data = pd.json_normalize(results_data['images'])
result_image_data.head()

In [ ]:
df_train = pd.merge(df_train_merge, result_image_data, left_on='image_id', right_on='id')
df_train.head()

In [ ]:

sns.set_theme(style="darkgrid")
sns.set(rc={'figure.figsize':(20,5)})
ax = sns.countplot(x="category_id", data=df_train)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.show()

In [ ]:
def plot_examples():
    fig, axs = plt.subplots(5, 4, figsize=(20, 20))
    fig.subplots_adjust(hspace = .1, wspace=.1)
    
    axs = axs.ravel()
    for i in range(20):
        img = cv2.imread(path+'train/'+df_train['file_name'][i])
        axs[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if len(df_train['detections'][i]) > 0:
            
            width = df_train['width'][i]
            height = df_train['height'][i]
            for dets in range(len(df_train['detections'][i])):
                bbox = df_train['detections'][i][dets]['bbox']            
                p = matplotlib.patches.Rectangle((width*bbox[0], height*bbox[1]),
                                              width*bbox[2],
                                              height*bbox[3],
                                              ec='r', fc='none', lw=2.)
                axs[i].add_patch(p)
                axs[i].set_title('category '+str(df_train['category_name'][i]))
        else:
            axs[i].set_title('no detections')
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
    plt.show()

In [ ]:
plot_examples()